In [1]:
%load_ext autoreload
%autoreload 2


import mahotas as mh
import numpy as np
import os
import time

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

import mlproof as mlp

In [2]:
PATCH_PATH = '/home/d/patches/cylinder_small1/'

In [3]:
def generate_patches(start_slice, end_slice):

    patch_index = 0

    all_patches_count = 0
    all_error_patches = []
    all_correct_patches = []
    
    for z in range(start_slice, end_slice):

        t0 = time.time()
        print 'working on slice', z
        input_image, input_prob, input_mask, input_gold, input_rhoana = mlp.Util.read_section('/home/d/data/cylinder/',z)

        error_patches, patches = mlp.Patch.patchify_maxoverlap(input_image, input_prob, input_mask, input_rhoana, input_gold, sample_rate=1)

        print 'Generated', len(error_patches), 'split error patches and', len(patches), ' correct patches in', time.time()-t0, 'seconds.'
        
        
        all_error_patches += error_patches
        all_correct_patches += patches
        
    
    
    NO_PATCHES = len(all_error_patches) + len(all_correct_patches)

    print 'We have a total of',NO_PATCHES,'patches.'
    print 'Errors:',len(all_error_patches)
    print 'Correct:',len(all_correct_patches)    
    
    PATCH_BYTES = 75*75
    P_SIZE = (NO_PATCHES, 1, 75,75) # rather than raveled right now
    
    p_image = np.zeros(P_SIZE, dtype=np.float32)
    p_prob = np.zeros(P_SIZE, dtype=np.float32)
    p_binary = np.zeros(P_SIZE, dtype=np.bool)
    p_merged_array = np.zeros(P_SIZE, dtype=np.bool)
#     p_dyn_obj = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
#     p_dyn_bnd = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
    p_border_overlap = np.zeros(P_SIZE, dtype=np.bool)
    p_larger_border_overlap = np.zeros(P_SIZE, dtype=np.bool)    
    p_target = np.zeros(NO_PATCHES)


    i = 0
    for p in all_error_patches:
        
        p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
        p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
        p_binary[i] = p['binary']#.ravel()
        p_merged_array[i] = p['merged_array']#.ravel()
#         p_dyn_obj[i] = p['dyn_obj'].ravel()
#         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
        p_border_overlap[i] = p['border_overlap']#.ravel()
        p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 1 # <--- important
        i += 1

        
    for p in all_correct_patches:

        p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
        p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
        p_binary[i] = p['binary']#.ravel()
        p_merged_array[i] = p['merged_array']#.ravel()
#         p_dyn_obj[i] = p['dyn_obj'].ravel()
#         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
        p_border_overlap[i] = p['border_overlap']#.ravel()
        p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 0 # <--- important
        i+=1
        
    
#     return p_image, p_prob, p_binary, p_merged_array, p_dyn_obj, p_dyn_bnd, p_border_overlap, p_larger_border_overlap, p_target
    return p_image, p_prob, p_binary, p_merged_array, p_border_overlap, p_larger_border_overlap, p_target



def shuffle_in_unison_inplace(a, b, c, d, e, f, g):
    assert len(a) == len(b)
    assert len(a) == len(g)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p], d[p], e[p], f[p], g[p]



def run(start_slice, end_slice, filename):
    
    if not os.path.exists(PATCH_PATH):
        os.makedirs(PATCH_PATH)
    
    p = generate_patches(start_slice, end_slice)
    
    shuffled = shuffle_in_unison_inplace(p[0],
                                         p[1],
                                         p[2],
                                         p[3],
                                         p[4],
                                         p[5],
                                         p[6]
                                        )
    
    print 'saving..'
    np.savez(PATCH_PATH+filename+'.npz', image=shuffled[0],
                                         prob=shuffled[1],
                                         binary=shuffled[2],
                                         merged_array=shuffled[3],
#                                          dyn_obj=shuffled[4],
#                                          dyn_bnd=shuffled[5],
                                         border_overlap=shuffled[4],
                                         larger_border_overlap=shuffled[5])
    np.savez(PATCH_PATH+filename+'_targets.npz', targets=shuffled[6])
    print 'Done!'
    
#     return correct, split, shuffled


In [4]:
run(40,50,'training')
run(60,62,'test')

working on slice 40
Loading z00000040.png
Generated 1132 split error patches and 1132  correct patches in 340.671875954 seconds.
working on slice 41
Loading z00000041.png
Generated 1220 split error patches and 1220  correct patches in 372.748060942 seconds.
working on slice 42
Loading z00000042.png
Generated 1208 split error patches and 1208  correct patches in 394.909323931 seconds.
working on slice 43
Loading z00000043.png
Generated 1052 split error patches and 1052  correct patches in 394.557433844 seconds.
working on slice 44
Loading z00000044.png
Generated 1204 split error patches and 1204  correct patches in 433.419236183 seconds.
working on slice 45
Loading z00000045.png
Generated 1236 split error patches and 1236  correct patches in 428.876406908 seconds.
working on slice 46
Loading z00000046.png
Generated 1108 split error patches and 1108  correct patches in 403.805125952 seconds.
working on slice 47
Loading z00000047.png
Generated 1244 split error patches and 1244  correct pa